# Final Project STA141B: 
## Do public scandals affect the viewer count of Popular Music Artists?

In [3]:
!pip install spotipy

In [4]:
import requests
import pandas as pd
import numpy as np
import sqlalchemy as sqla
# import requests_cache
import plotnine as p9
import time
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [5]:
client_id = '65580595dc324a1cb2e890e9800479c4'
client_secret = '20a9994294354eae8ef6cc4ea75e48a3'

client_credentials_manager = SpotifyClientCredentials(client_id,client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [7]:
username = 'spotify:user:ef8zswxc6q41041caiq7x28pa'
playlist_id = 'spotify:playlist:4FLeoROn5GT7n2tZq5XB4V?offset=101'
#album_id = 'spotify:album:0jKnYgav4YIHNyzfpm5eoF'

In [41]:
def show_tracks(results,uriArray):
    for item in results["items"]:
        track = item["track"]
        uriArray.append(track["id"])

def get_playlist_track_id(username, playlist_id):
    trackId = []
    results = sp.user_playlist(username, playlist_id)
    tracks = results['tracks']
    show_tracks(tracks,trackId)
    while tracks['next']: # checks whether there is a next page for this playlist
        tracks = sp.next(tracks)
        show_tracks(tracks, trackId)
    return trackId

var = get_playlist_track_id(username, playlist_id) # 205 tracks in TikTok playlist

In [114]:
# extract all features of all (over 100) songs on a playlist into a dataframe
def playlist_to_df(username, playlist_id):
    # get JSON of playlist metadata for first page of songs
    spotipy_response = sp.user_playlist_tracks(username,playlist_id)['tracks']
    
    # record track information
    results = spotipy_response['items']
    
    # while there is a next page of songs (100 songs per page)
    while spotipy_response['next']:
        # replace previous page information with new page
        spotipy_response = sp.next(spotipy_response)
        
        # add track information to list
        results.extend(spotipy_response['items']) 
    
    first = True
    
    # loop through tracks
    for item in results:
        try:
            track = item['track']
            
            # extract id and other features from track metadata and store in dictionary
            ids = track['id']
            
            artist_list = []
            for artist in track['artists']:
                artist_list = artist['name']

            new_row = {'id':[track['id']], 'title':[track['name']], 'all_artists':[artist_list],
                       'popularity':[track['popularity']], 
                       'release_date':[track['album']['release_date']]}

            # extract given features for track and add to dictionary
            features = sp.audio_features(ids)[0]
            
            for key in features:
                new_row[key] = [features[key]]
            
            # turn dictionary into dataframe
            new_df = pd.DataFrame(new_row)
            
            # create new dataframe or add to existing dataframe
            if first:
                features_df = new_df
                first = False
            else:
                features_df = pd.concat([features_df, new_df], ignore_index = True)
                
        except:
            continue
            
    # drop unnecessary features
    features_df = features_df.drop(['speechiness', 'type', 'uri', 'track_href', 'analysis_url'], axis=1)
    
    # return dataframe with all songs and desired features
    return features_df

In [115]:
df = playlist_to_df(username, playlist_id)
df.to_csv("TikTok_Playlist.csv",index=False)

In [116]:
df

,id,title,all_artists,popularity,release_date,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,5reQI13tWWYDLMrGcUF4Mk,Nuestra Canción,Vicente Garcia,83,2015-06-16,0.906,0.731,6,-7.021,0,0.42600,0.000005,0.0853,0.794,106.995,260080,4
1,6M47gaKejso9772SKTa3yH,Face Off,Dwayne Johnson,81,2021-10-08,0.693,0.935,8,-4.323,1,0.18000,0.000000,0.1790,0.391,80.882,219802,4
2,4IhTXiZLKATmwhMZIb1GQN,Lights Are On,Tom Rosenthal,78,2018-10-26,0.440,0.040,3,-16.887,1,0.98100,0.465000,0.1110,0.249,135.965,193749,4
3,15HMh4yxdf4wyxSZSlOgGZ,SAD GIRLZ LUV MONEY Remix (feat. Kali Uchis an...,Moliy,88,2021-09-16,0.859,0.525,1,-8.051,0,0.43400,0.000023,0.1080,0.312,110.066,204764,4
4,0gplL1WMoJ6iYaPgMCL0gX,Easy On Me,Adele,100,2021-10-14,0.604,0.366,5,-7.519,1,0.57800,0.000000,0.1330,0.130,141.981,224695,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,0ecC8p17cDNlxHXkuYqeR6,Surrender,Natalie Taylor,76,2019-10-29,0.390,0.448,5,-8.570,0,0.80000,0.182000,0.1070,0.077,144.890,183103,5
201,6otiaV2fagE3s8IvP6WkwG,I'm Just a Kid,Simple Plan,73,2018-04-06,0.527,0.834,1,-5.531,1,0.00107,0.000102,0.0993,0.422,110.065,198333,4
202,6vFsBXYczYsP0H3lgunZOm,Cloud 9,Beach Bunny,80,2020-02-14,0.637,0.929,4,-3.593,1,0.00167,0.000026,0.3440,0.902,80.974,147351,4
203,11ZulcYY4lowvcQm4oe3VJ,Slumber Party (feat. Princess Nokia),Princess Nokia,79,2021-01-15,0.964,0.398,11,-8.981,0,0.00151,0.000039,0.1010,0.563,105.012,178405,4
